In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
import numpy as np
import string
import random
from SoundsLike.SoundsLike import Search
import Levenshtein

In [3]:
#df = pd.read_csv("../../data/conll03/train.txt", sep=" ")
df = pd.read_csv("../../data/conll03/dev.txt", sep=" ")
#df = pd.read_csv("data/test.txt", sep=" ")

In [4]:
df.head()

,-DOCSTART-,-X-,-X-.1,O
0,CRICKET,NNP,B-NP,O
1,-,:,O,O
2,LEICESTERSHIRE,NNP,B-NP,B-ORG
3,TAKE,NNP,I-NP,O
4,OVER,IN,B-PP,O


In [5]:
df = df[df['O'].notna()]

In [6]:
df.drop(['-X-', '-X-.1'], axis=1, inplace=True)

In [7]:
df.columns = ['Word', 'Tag']

In [8]:
df.head()

,Word,Tag
0,CRICKET,O
1,-,O
2,LEICESTERSHIRE,B-ORG
3,TAKE,O
4,OVER,O


In [9]:
#df.drop(df[df['Word'] == "-DOCSTART-"].index, axis=0, inplace=True)

In [10]:
df['Tag'].unique()

array(['O', 'B-ORG', 'B-LOC', 'B-MISC', 'I-MISC', 'B-PER', 'I-PER',
       'I-LOC', 'I-ORG'], dtype=object)

In [11]:
tags = df['Tag'].values
words = df['Word'].values

In [12]:
new_tags = []
for t in tags:
    if t == "B-ORG":
        new_tags.append("ORG")
    elif t == "B-PER":
        new_tags.append("PER")
    elif t == "I-PER":
        new_tags.append("PER")
    elif t == "B-LOC":
        new_tags.append("LOC")
    elif t == "I-ORG":
        new_tags.append("ORG")
    elif t == "I-LOC":
        new_tags.append("LOC")
    else:
        new_tags.append("O")

In [13]:
df.drop(['Tag'], axis=1, inplace=True)
df.drop(['Word'], axis=1, inplace=True)

In [14]:
df['Tag'] = new_tags
df['Word'] = words

In [15]:
len(df)

50937

In [16]:
sentence_no = 0
sentence = []
for w in words:
    sentence.append(sentence_no)
    if w == ".":
        sentence_no = sentence_no + 1

In [17]:
df.insert(0, 'Sentence #', sentence)

In [18]:
df.head()

,Sentence #,Tag,Word
0,0,O,CRICKET
1,0,O,-
2,0,ORG,LEICESTERSHIRE
3,0,O,TAKE
4,0,O,OVER


In [19]:
df['Tag'].unique()

array(['O', 'ORG', 'LOC', 'PER'], dtype=object)

In [20]:
len(df)

50937

In [21]:
dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if tag != "O":
        error = []
        try:
            x = Search.perfectHomophones(word.lower())
            #print(x)
            for d in x:
                err = Levenshtein.distance(word.lower(), d)
                error.append(err)
                word = x[error.index(max(error))]
        except ValueError:
            pass 
        dataset.append((sentence_no, tag, word))
    else:
        dataset.append((sentence_no, tag, word))
new_df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

In [22]:
new_df[new_df['Tag']!="O"][50:60]

,Sentence #,Tag,Word
370,11,ORG,Derbyshire
398,12,ORG,Kent
405,12,ORG,Nottinghamshire
418,13,PER,Pol
419,13,PER,Jonson
424,13,PER,Martyn
425,13,PER,Maciag
433,14,ORG,Kent
448,16,LOC,Lunden
464,16,LOC,Lester


In [23]:
broken_df = new_df

In [35]:
broken_df['Original Word'] = df['Word']

In [36]:
broken_df.drop('original_word', axis=1, inplace=True)

In [46]:
broken_df[broken_df['Tag']!="O"][50:60]

,Sentence #,Tag,Word,Original Word
370,11,ORG,Derbyshire,Derbyshire
398,12,ORG,Kent,Kent
405,12,ORG,Nottinghamshire,Nottinghamshire
418,13,PER,Pol,Paul
419,13,PER,Jonson,Johnson
424,13,PER,Martyn,Martin
425,13,PER,Maciag,McCague
433,14,ORG,Kent,Kent
448,16,LOC,Lunden,LONDON
464,16,LOC,Lester,Leicester


In [ ]:
df.to_csv("../../data/conll03/new_conll_dev_preprocessed_with_punctuation_with_broken_entities2.csv")

In [ ]:
x = Search.perfectHomophones("london")

In [ ]:
x

In [ ]:
error = []
for d in x:
    err = Levenshtein.distance("london", d)
    error.append(err)
word = x[error.index(max(error))]

In [ ]:
word